In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import sys
import os
sys.path.append(os.pardir)
import gc
import pdb
import numpy as np
import torch
from torch.utils.data import DataLoader
from data.dataset import songbird_full_spectrogram
from configs.cfg import *

In [3]:
from train.train_recurrent import train, evaluate
from models.nets_recurrent import RecurrentNetv1

In [4]:
opts_dict = {'training_path': os.path.join(EXT_PATH,'id_list_train.pkl'), 'outf': SAVE_PATH, 'external_file_path': EXT_PATH,
             'test_path':os.path.join(EXT_PATH,'id_list_test.pkl'),
        'age_weights_path': '', 
       'distance_fun': 'L1', 'subset_age_weights' : [0., 1.], 'workers': 6, 'batchSize': 128, 
        'imageH': 129, 'imageW': 16, 'noverlap':0, 'nz': 16, 'nepochs': 50, 'log_every': 500,
        'checkpoint_models':  True, 'ksteps_ahead': 5, 
       'lr': 1e-5, 'l2': 0.0, 'schedule_lr':False, 'beta1': 0.5, 'cuda': True, 'ngpu': 1,
        'log_every': 300, 'sample_rate': 16000.,'noise_dist': 'normal','z_var': 1.,
        'nfft': 256, 'get_audio': False,'manualSeed': [], 'do_pca': True, 
        'num_rnn': 200, 'num_layers': 2, 'dropout': 0.1, 'num_linear':200,
        'bidirectional':True, 'leak': 0.1, 'max_length':400}

# setup datasets and dataloaders

In [13]:
# initialize the dataset and dataloader objects
train_dataset = songbird_full_spectrogram(opts_dict['training_path'], opts_dict['external_file_path'], 
                                          opts_dict['subset_age_weights'], opts_dict['max_length'])

train_dataloader = DataLoader(train_dataset, batch_size=opts_dict['batchSize'], sampler = None,
                                         shuffle=True, num_workers=int(opts_dict['workers']),
                                    drop_last = True)

test_dataset = songbird_full_spectrogram(opts_dict['test_path'], opts_dict['external_file_path'],
                               opts_dict['subset_age_weights'], opts_dict['max_length'])

test_dataloader = DataLoader(test_dataset, batch_size= opts_dict['batchSize'],
                                         shuffle=True, num_workers=int(opts_dict['workers']),
                            drop_last = True)


In [14]:
len(train_dataset)

685129

In [15]:
xx, age = train_dataset.__getitem__(1050)

In [16]:
xx.shape

torch.Size([129, 400])

In [17]:
age

tensor([0.4100])

# setup model and test it

In [18]:
# instantiate network
model =  RecurrentNetv1(Hin=opts_dict['imageH'], nz=opts_dict['imageH'],
                        nrnn=opts_dict['num_rnn'], nlin=opts_dict['num_linear'], 
                        nlayers=opts_dict['num_layers'], bidirectional= opts_dict['bidirectional'], 
                        dropout=opts_dict['dropout'], leak=opts_dict['leak'])

model = model.cuda()


In [66]:
del model
gc.collect()

297

In [69]:
# test on some input
xx = torch.cuda.FloatTensor(32,100,129)

yy = model(xx)
yy.shape

torch.Size([32, 100, 129])

# train

In [ ]:
# train
model, train_loss = train(model, train_dataloader, test_dataloader, opts_dict)